In [71]:
import pandas as pd
import scipy


from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Implementation


### Genetic Algorithm

In [274]:
class genetic_algorithm():#, p_cross=0.5, p_mut=0.5, p_inv=0.5, eps=1e-5):
    import random
    import math
    import numpy as np
    
    def __init__(self, f, low, high, eps=1e-2):
        self.f = f
        self.low = low
        self.high = high
        self.eps = eps
        self.d = self.high - self.low
        k = int(self.d / self.eps + 1)
        gen_len = int(math.log(k, 2)) + 1
        self.k = 2 ** gen_len + 1
        
        self._create_sets()
   

    def _create_sets(self):
        """
        create phenotype, integer represantation of phenotype and genotype
        """
        self.phenotype = np.linspace(self.low, self.high, self.k)
        self.phenotype_int = dict(zip(self.phenotype, range(self.k)))
        
        self.gen_len = int(math.log(self.k, 2)) + 1
        self.genotype = np.array([self._int_to_bin(el) for el in self.phenotype_int.values()])
      
    
    def _int_to_bin(self, int_number):
        """
        return a binary represantation of an integer
        """
        return str(bin(int_number)[2:]).zfill(self.gen_len)
        
    def _bin_to_int(self, bin_number):
        """
        return an integer decimal represantation of binary
        """
        return int(str(bin_number), 2)
   
    def _float_to_int(self, float_number):
        """
        return integer representation of float (index of phenotype)
        """
        return self.phenotype_int[float_number]
    
    def _int_to_float(self, int_number):
        """
        return float of as a phenotype in index
        """
        return self.phenotype[min(int_number,self.k-1)]

    def _float_to_bin(self, float_number):
        """
        return composition of _float_to_int & _int_to_bin
        """
        return self._int_to_bin(self._float_to_int(float_number))
    
    def _bin_to_float(self, bin_number):
        """
        return composition of _bin_to_int & _int_to_float
        """
        return self._int_to_float(self._bin_to_int(bin_number))
   
    def _find_nearest_element(self, element):
        """
        find nearest element in phenotype to the element
        :param: element - float
        return float element
        """
        return self.phenotype[argmax(-abs(self.phenotype - element))]

    def _evaluate_fitness(self):
        """
        evaluate fitness-function of each element from generation
        """
        self.fitness_of_generation = np.array([self.f(el) for el in self.current_generation])

    def selection(self):
        """
        select randomly two elements from elements with fitness-function > mean
        create current_parents as an array of two floats
        """
        self._evaluate_fitness()
        m = self.fitness_of_generation.mean()
        #population = np.extract(self._evaluate_fitness() > m, self.current_generation)
        indices = np.where(self.fitness_of_generation > m)[0]
        random.shuffle(indices)
        self.parents = self.current_generation[indices[:2]] 
        if self.print_out:
            print('-> selected parents: ', self.parents)
            print('-> selected parents (int): ', [self.phenotype_int[p] for p in self.parents])

    def crossover(self, p=0.5):
        """
        implement 1-point crossover
        create two childs from two parents
        """
        #n_coord = selected.shape[1]
        #new_element = np.zeros(n_coord)
        #for i in range(n_coord):
        #    new_element[i] = np.random.choice(selected[:,i], p=[p, 1 - p])
        #    print(selected[:,i])
        parent0_bin = self._float_to_bin(self.parents[0])
        parent1_bin = self._float_to_bin(self.parents[1])
        if self.print_out:
            print('-> selected parents (binary): ', [parent0_bin, parent1_bin])
        point = np.random.randint(self.gen_len) + 1
        if self.print_out:
            print('-> crossover point: ', point)
        self.left_child = parent0_bin[:point] + parent1_bin[point:]
        self.right_child = parent1_bin[:point] + parent0_bin[point:]
        if self.print_out:
            print('-> created childs (binary): ', [self.left_child,
                                               self.right_child])
        self.left_child = self._bin_to_float(self.left_child)
        self.right_child = self._bin_to_float(self.right_child)
        if self.print_out:
            print('-> created childs (int): ', [self.phenotype_int[self.left_child],
                                      self.phenotype_int[self.right_child]])
        
            print('-> created childs: ', [self.left_child, self.right_child])
        
        
    def mutation(self, element, p=0.01):
        """
        inverse each bit of element with probability p
        :param: element - float
        return mutated element as float
        """
        if self.print_out:
            print('-> element: ', element)
        element_bin = self._float_to_bin(element)
        if self.print_out:
            print('-> element (binary): ', element_bin)
        mutated_bin = ''.join(list(map(lambda x: str(1 - int(x))
                                       if np.random.choice([0,1], p=[p, 1 - p]) == 0
                                       else x, element_bin)))
        if self.print_out:
            print('-> mutated element (binary): ', mutated_bin)
        mutated = self._bin_to_float(mutated_bin)
        mutated = self._find_nearest_element(mutated)
        if self.print_out:
            print('-> mutated element: ', mutated)
        return mutated
    
    def mutation2(self, element, p=0.01):
        """
        inverse each bit of element with probability p
        :param: element - a binary represantation, of type list i.e. [0,1,0,1,1]
        return a list which is a binary represantation of mutated element
        """
        mutated = list(map(lambda x: 1 - x 
                           if np.random.choice([0,1], p=[p, 1 - p]) == 0
                           else x, element))
        return mutated

    def mutation3(self, element, p=0.01):
        """
        inverse each bit of element with probability p
        :param: element - a binary represantation, of type string i.e. '01011'
        return a string which is a binary represantation of mutated element
        """
        mutated = ''.join(list(map(lambda x: str(1 - int(x))
                                   if np.random.choice([0,1], p=[p, 1 - p]) == 0
                                   else x, element)))
        return mutated
    
    def inversion(self, element, p=0.5):
        return
    
    
    def run_algorithm(self, p_cross=0.5, p_mut=0.01,
                            p_inv=0.5, print_out=True):
        
        self.print_out = print_out
    
        self.current_generation = self.phenotype
        self.new_generation = []
        if self.print_out:
            print('-> current generation:', self.current_generation)
        #while not(stop_condition(fits)):
        for i in range(self.k):
            self.selection()
            self.crossover(p_cross)
            self.left_child  = self.mutation(self.left_child,  p_mut)
            self.right_child = self.mutation(self.right_child, p_mut)
            #self.left_child  = self.inversion(self.left_child,  p_inv)
            #self.right_child = self.inversion(self.right_child, p_inv)
            self.new_generation.append(self.left_child)
            self.new_generation.append(self.right_child)
        if self.print_out:
            print('-> new generation:', list[set(self.new_generation)])
        self.current_generation = self.new_generation
        self._evaluate_fitness()
        
        return self.current_generation[self.fitness_of_generation.argmax()], self.fitness_of_generation.max()

In [280]:
f = lambda x: 2 * x - x**3 + (1 + x)**.5  + 1
low = 0
high = 10
GA = genetic_algorithm(f, low, high, eps=0.01)
GA.phenotype
GA.phenotype_int
GA.genotype
GA.gen_len
GA.k
x0, f0 = GA.run_algorithm(print_out=False)
print('x0 = ', x0)
print('f0 = ', f0)

x0 =  0.888671875
f0 =  3.44981565533


In [171]:

pops = init(k, 2, low, high)
fits = evaluate(f, pops)
sel = select(pops)
pops, fits, sel, crossover(sel)

[ 4.3597285   7.00445783]
[ 3.34535538  1.04766183]


(array([[ 4.66029837,  7.40687085],
        [ 4.91780452,  7.68397795],
        [ 4.3597285 ,  3.34535538],
        [ 7.38964352,  0.95294612],
        [ 7.00445783,  1.04766183]]),
 array([ 76.58011661,  83.22831845,  30.19863521,  55.51493765,  50.16002487]),
 array([[ 4.3597285 ,  3.34535538],
        [ 7.00445783,  1.04766183]]),
 array([ 4.3597285 ,  3.34535538]))

### Linear regression model

In [73]:
def linear_prediction(w, X):
    """
    Returns a linear regression model prediction labels for objects in matrix 
    X using weights w:
    y_pred = (X,w)
    """
    if X.ndim == 1:
        return(np.insert(X, 0, 1).dot(w))
    else:
        n = X.shape[0]
        return np.dot(np.hstack((np.ones((n,1)),X)),w)

def mean_squared_error(y, y_pred):
    """
    Returns a mean squared error between real and predicted labels
    """
    y = np.array(y)
    y_pred = np.array(y_pred)
    mse = np.sum((y - y_pred)**2) / y.size
    return mse

def mean_absolute_error(y, y_pred):
    """
    Returns a mean absolute error between real and predicted labels
    """
    y = np.array(y)
    y_pred = np.array(y_pred)
    mae = np.sum(abs(y - y_pred)) / y.size
    return mae

def cost_function(w, X, y, type_f='mse'):
    """
    Returns a cost function of a linear model with coefficients w
    oh features X and labels y.
    """
    if (type_f == 'mae'):
        return mean_absolute_error(y, linear_prediction(w, X))
    elif (type_f == 'mse'):
        return mean_squared_error(y, linear_prediction(w, X))
    else:
        #raise
        print('error: incorrect type of function')
        return

def linear_regression_fit(X, y, minimize='grad_desc', cost_f='mse',
                          w0=None, eta=1e-2):
    """
    Returns weights that minimizes cost function.
    """   
    if (minimize == 'analytical'):
        # the cost function is 'mse' automatically
        # X_t = X.transpose()
        # w = np.linalg.inv(X_t.dot(X)).dot(X_t).dot(y)
        n = X.shape[0]
        X = np.hstack((np.ones((n,1)),X))
        w = np.linalg.lstsq(X,y)[0]
        
    elif (minimize == 'grad_desc'):
        # the cost function is 'mse' automatically
        if(w0 == None):
            w0 = np.zeros(X.shape[1] + 1)
        n = X.shape[0]
        X = np.hstack((np.ones((n,1)),X))
        w = gradient_descent(X, y, w0, eta)[0]
        w = w.reshape(w.shape[0])

    elif (minimize == 'st_grad_desc'):
        # the cost function is 'mse' automatically
        if(w0 == None):
            w0 = np.zeros(X.shape[1] + 1)
        n = X.shape[0]
        X = np.hstack((np.ones((n,1)),X))
        w = stochastic_gradient_descent(X, y, w0, eta)[0]
        w = w.reshape(w.shape[0])
    
    elif (minimize == 'scipy_minimize'):
        if(w0 == None):
            w0 = np.zeros(X.shape[1] + 1)
        w = scipy.optimize.minimize(lambda w: cost_function(w, X, y, cost_f), w0).x
    else:
        #raise
        print('error: incorrect minimization method')
        return
    return w

### Gradient descent

In [74]:
def gradient_step(X, y, w, eta=0.01):
    grad = 2 * X.T.dot(X.dot(w) - y) / X.shape[0] 
    w_next = w - eta * grad 
    return w_next

def gradient_descent(X, y, w0=None, eta=1e-2,
                     max_iter=1e4, min_weight_dist=1e-8):    
    X = np.array(X)
    y = np.array(y)
    if(w0 == None):
        w0 = np.zeros(X.shape[1])
        
    w = w0
    weight_dist = np.inf
    errors = []
    iter_num = 0
        
    while weight_dist > min_weight_dist and iter_num < max_iter: 
        w_next = gradient_step(X, y, w, eta)        
        errors.append(mean_squared_error(y, X.dot(w_next)))
        weight_dist = np.linalg.norm(w_next - w)      
        w = w_next
        iter_num += 1
    return w, errors

### Stochastic gradient descent

In [75]:
def stochastic_gradient_step(X, y, w, k, eta=0.01):
    grad = 2 * (X[k].dot(w) - y[k]) * X[k,:] / X.shape[0] 
    w_next = w - eta * grad   
    return w_next

def stochastic_gradient_descent(X, y, w0=None, eta=1e-2, max_iter=1e4,
                                min_weight_dist=1e-8, seed=42):    
    X = np.array(X)
    y = np.array(y)
    if(w0 == None):
        w0 = np.zeros(X.shape[1])
        
    w = w0
    weight_dist = np.inf
    errors = []
    iter_num = 0
    #import random
    #k = np.array(range(X.shape[0]))
    #random.shuffle(k)
    np.random.seed(seed)
        
    while weight_dist > min_weight_dist and iter_num < max_iter:  
        #print('\nit.',iter_num)
        #print('w: ',w)
        random_ind = np.random.randint(X.shape[0])
        w_next = stochastic_gradient_step(X, y, w, random_ind, eta)     
        errors.append(mean_squared_error(y, X.dot(w_next)))
        weight_dist = np.linalg.norm(w_next - w)      
        #print('d: ',weight_dist)
        w = w_next
        iter_num += 1
    return w, errors

### Auxiliary functions

In [76]:
def print_result(coef, true, predict, cut = 5):
    print('w:\n',coef,'\n')
    print('true vs. prediction:\n',np.vstack((true,predict)).T[:cut],'\n...')
    print('root mean squared error: ',round((mean_squared_error(true, predict))**.5,3))
    print('mean absolute error: ',round(mean_absolute_error(true, predict),3))

In [77]:
def plot_for_one_feature(train_data, train_labels, w, title):
    x = np.linspace(train_data.min(), train_data.max(), 2).reshape((2,1))
    plt.figure(figsize = (8,5))
    plt.plot(train_data, train_labels, 'o', markersize = 3)
    #print(x)
    #print(linear_prediction(w, x))
    plt.plot(x, linear_prediction(w,x), '-', linewidth = 4)
    plt.xlabel('feature')
    plt.ylabel('label')
    plt.title(title)
    plt.show()

In [78]:
def normalize(X, mean_std=True):
    if mean_std:
        means, stds = X.mean(axis=0), X.std(axis=0)
        X = (X - means) / stds
    else:
        minim, maxim = data.min(axis = 0), data.max(axis = 0)
        X = (X - minim) / (maxim - minim)
    return X

# Data

### Generate dataset

In [1]:
from sklearn import datasets 

sample_size = 200
data, target = datasets.make_regression(n_samples = sample_size,
                                        n_features = 1, 
                                        n_informative = 1, 
                                        n_targets = 1, noise = 5.,
                                        coef = False, random_state = 2)

#data = normalize(data)

array([  75.87807753,  -25.62849436,   -8.52830532,   34.7295849 ,
         14.94573823,  -43.80916304,  -18.58567559,  -16.07885396,
         12.29620556,   -2.59927619,  -37.8086868 ,  -12.76903779,
        -27.47018765,  -27.61324039,  -17.26011438,   26.52165317,
          1.73192444,   44.15478142,  -14.00371474,   39.03303189,
         49.8749579 ,   34.47979933,   15.80407304,  -37.78326744,
        -75.63811301,   17.94076279,  -58.97860917,   36.61443886,
         36.61808062,   -2.8224484 ,   -0.69879232,   47.26971358,
        104.46237473,   36.82761358,  -44.39979228,  -15.00089409,
        -28.46821439,  -48.48269864,  -27.05864665,  -40.52520996,
         -4.50734694,  -50.21672081,   -7.02962287,    7.4786154 ,
          8.34655722,   -6.19297489,    5.31972292,   55.88383884,
         16.43721584,   94.1177299 ,   42.37765206,  -38.65087801,
         86.82359442,   18.55162032,  -28.31097045,   44.52145512,
         -7.38026943,    9.49211339,   45.78374099,  -13.52407

### Dataset "Advertising"

In [80]:
#data = pd.read_csv('Data/weights_heights.csv', index_col='Index')
data_frame = pd.read_csv('Data/advertising.csv')

features = ['TV'] # , 'Radio','Newspaper']
labels = ['Sales']
data = np.array(data_frame[features].values, dtype=float)[:sample_size]
target = np.array(data_frame[labels].values, dtype=float)[:sample_size]
target = target.reshape(target.shape[0])

data = normalize(data)

### Toy dataset "Boston"

In [81]:
dataset = datasets.load_boston()
data = dataset.data
target = dataset.target

data = normalize(data)

### Split dataset into train & test samples

In [82]:
from sklearn import cross_validation as cross_val

train_data, test_data, \
train_labels, test_labels = cross_val.train_test_split(data, target,
                                                       test_size = 0.3)

In [83]:
print('train_data: \n',train_data[:5],'\n...\n')
print('train_labels: \n',train_labels[:5],'...')

train_data: 
 [[-0.40766012  0.97154295 -0.73637217 -0.27259857 -1.05124209  0.2996992
  -1.78424818  0.80653853 -0.29308074 -0.47061187 -1.08911039  0.29533561
  -0.55832104]
 [-0.40354288 -0.48772236 -0.37597609 -0.27259857 -0.29970737  0.26978136
   1.01436883 -0.6475206  -0.52300145 -0.14395131  1.13022958  0.4228511
  -0.05369542]
 [ 1.07222115 -0.48772236  1.01599907 -0.27259857  1.60072524 -0.61350702
   0.99658854 -0.90293643  1.66124525  1.53092646  0.80657583 -1.27355443
   1.56110656]
 [-0.20492927 -0.48772236  1.2319449   3.66839786  0.43455068  2.16172808
   1.05348546 -0.83396037 -0.52300145 -0.03110494 -1.73641788  0.36112176
  -1.50449408]
 [-0.41115675 -0.48772236  0.11573841 -0.27259857  0.15812412  0.43931575
   0.01867281 -0.62579623 -0.98284286 -0.80321172  1.17646583  0.38721693
  -0.41814726]] 
...

train_labels: 
 [ 26.4  19.8  10.8  50.   22.4] ...


# Run models and output

### Analytical OLS method

In [84]:
w = linear_regression_fit(train_data, train_labels, minimize='analytical')
print_result(w,test_labels,linear_prediction(w, test_data))
if train_data.shape[1] == 1:
    #plot_for_one_feature(train_data, train_labels, w, 'train')
    plot_for_one_feature(test_data, test_labels, w, 'test')

w:
 [ 22.44376062  -0.49832497   1.1037074    0.36817348   1.12305293
  -1.86223667   3.39902492  -0.30222417  -2.83374334   2.1979636
  -2.03170579  -1.85956253   0.93330698  -3.16650413] 

true vs. prediction:
 [[ 18.2         18.82826868]
 [ 22.          26.08788447]
 [ 13.4         16.39740984]
 [ 30.8         31.26097619]
 [ 24.3         23.43286557]] 
...
root mean squared error:  5.448
mean absolute error:  3.546


### Numerical method using gradient descent

In [85]:
w0 = np.array([1,0])
w = linear_regression_fit(train_data, train_labels,
                          minimize='grad_desc', eta=0.1)

print_result(w,test_labels,linear_prediction(w, test_data))
if train_data.shape[1] == 1:
    #plot_for_one_feature(train_data, train_labels, w, 'train')
    plot_for_one_feature(test_data, test_labels, w, 'test')

w:
 [ 22.44376062  -0.49832493   1.10370736   0.36817327   1.12305296
  -1.86223661   3.39902497  -0.30222419  -2.83374335   2.19796316
  -2.03170529  -1.8595625    0.93330697  -3.16650412] 

true vs. prediction:
 [[ 18.2         18.82826874]
 [ 22.          26.08788453]
 [ 13.4         16.39740995]
 [ 30.8         31.26097613]
 [ 24.3         23.43286543]] 
...
root mean squared error:  5.448
mean absolute error:  3.546


/Users/Yaroslava/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  # Remove the CWD from sys.path while we load stuff.


### Numerical method using stochastic gradient descent

In [86]:
w0 = np.array([1,0])
w = linear_regression_fit(train_data, train_labels,
                          minimize='st_grad_desc', eta=0.05)

print_result(w,test_labels,linear_prediction(w, test_data))
if train_data.shape[1] == 1:
    #plot_for_one_feature(train_data, train_labels, w, 'train')
    plot_for_one_feature(test_data, test_labels, w, 'test')

/Users/Yaroslava/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  # Remove the CWD from sys.path while we load stuff.


w:
 [ 21.05828054  -0.44114324   0.90649008  -0.39014135   1.02538009
  -0.86013865   3.59852964   0.1829777   -1.66467708   0.34865592
  -0.42275516  -1.71221661   0.66769586  -2.78834143] 

true vs. prediction:
 [[ 18.2         18.51560566]
 [ 22.          24.85790143]
 [ 13.4         16.76402607]
 [ 30.8         28.65614294]
 [ 24.3         21.26785382]] 
...
root mean squared error:  5.874
mean absolute error:  3.756


### Numerical method using scipy.optimize.minimize

In [87]:
w0 = np.array([1,0])
w = linear_regression_fit(train_data, train_labels, minimize='scipy_minimize', cost_f='mae')

w.reshape((1,w.shape[0]))
print_result(w,test_labels,linear_prediction(w, test_data))
if train_data.shape[1] == 1:
    #plot_for_one_feature(train_data, train_labels, w, 'train')
    plot_for_one_feature(test_data, test_labels, w, 'test')

w:
 [ 21.57250514  -0.25650434   0.61159515   0.21744535   0.38726468
  -0.81356805   3.97613022  -0.70018654  -1.83983897   1.08398396
  -1.79501132  -1.61558548   1.0326189   -2.54325451] 

true vs. prediction:
 [[ 18.2         18.83540185]
 [ 22.          25.86114687]
 [ 13.4         14.83382662]
 [ 30.8         29.77006751]
 [ 24.3         22.13718297]] 
...
root mean squared error:  5.706
mean absolute error:  3.508


### sklearn.linear_model.SGDRegression for check

In [88]:
from sklearn import linear_model 

sgd_regressor = linear_model.SGDRegressor(random_state=None, n_iter=20)
sgd_regressor.fit(train_data, train_labels)
sgd_regressor.predict(test_data)
w = [sgd_regressor.intercept_[0]]
w.extend(sgd_regressor.coef_)
w = np.array(w)
print_result(w,test_labels,sgd_regressor.predict(test_data))
if train_data.shape[1] == 1:
    #plot_for_one_feature(train_data, train_labels, w, 'train')
    plot_for_one_feature(test_data, test_labels, w, 'test')

w:
 [ 22.44025172  -0.32870933   0.91243119  -0.04248737   1.21194893
  -1.47927849   3.54029591  -0.20553943  -2.55304485   1.26651369
  -0.98148952  -1.73279411   0.91530178  -3.15202024] 

true vs. prediction:
 [[ 18.2         19.16797719]
 [ 22.          26.28505887]
 [ 13.4         17.20411417]
 [ 30.8         30.52080577]
 [ 24.3         22.95317484]] 
...
root mean squared error:  5.527
mean absolute error:  3.564


/Users/Yaroslava/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


# TRASH

In [163]:
def init(k, dim, low, high):
    """
    initialize population of k elements of dimension = dim
    low is the lowest value of possible element
    high is the highest value of possible element
    """
    t = 0
    init_population = np.array([np.array((high[i] - low[i]) * \
                                         random.sample(dim) + low[i])
                                for i in range(k)])
    return init_population

def evaluate(fit, population):
    """
    evaluate fitness-function of each element from population
    """
    return np.array([fit(element) for element in population])

def select(population):
    """
    randomly select two elements from population
    """
    indices = np.array(range(population.shape[0]))
    random.shuffle(indices)
    indices = indices[:2]
    return population[indices]

def genotype(phenotype):
    """
    return genotype as a binary represantation of phenotype
    """
    return int(bin(phenotype)[2:])

def phenotype(genotype):
    """
    return phenotype as a decimal represantation of genotype
    """
    return int(str(genotype), 2)

def crossover(selected, p=0.5):
    n_coord = selected.shape[1]
    new_element = np.zeros(n_coord)
    for i in range(n_coord):
        new_element[i] = np.random.choice(selected[:,i], p=[p, 1-p])
        print(selected[:,i])
    return new_element

def mutation(selected, p=0.5):
    return

def inversion(selected, p=0.5):
    return

def stop_condition(f):
    return

